In [1]:
import os
import sys
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from mpl_toolkits.mplot3d import Axes3D # for 3d plotting
from sklearn.model_selection import train_test_split
 
%matplotlib inline

Using TensorFlow backend.


In [2]:
def start_config(gpu):
        print('Configuring environment')

        tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
        #tf.reset_default_graph()

        os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# load the data
with h5py.File('../data/full_dataset_vectors.h5', 'r') as hf:
    x_train_raw = hf["X_train"][:]
    y_train_raw = hf["y_train"][:]
    x_test_raw = hf["X_test"][:]
    y_test_raw = hf["y_test"][:]


# length check
assert(len(x_train_raw) == len(y_train_raw))
assert(len(x_test_raw) == len(y_test_raw))

In [4]:
# 1D vector to rgb values, provided by ../input/plot3d.py
def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:,:-1]

# Transform data from 1d to 3d rgb
def rgb_data_transform(data):
    data_t = []
    for i in range(data.shape[0]):
        data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
    return np.asarray(data_t, dtype=np.float32)

In [5]:
n_classes = 10 # from 0 to 9, 10 labels totally

x_train = rgb_data_transform(x_train_raw)
x_test = rgb_data_transform(x_test_raw)

y_train = to_categorical(y_train_raw, n_classes)
y_test = to_categorical(y_test_raw, n_classes)


x_train = x_train[:,0:2,:,:]
print('x_train:'+str(x_train.shape))

x_test = x_test[:,0:2,:,:]
print('x_test:'+str(x_test.shape))

x_train:(10000, 2, 16, 16, 3)
x_test:(2000, 2, 16, 16, 3)


In [6]:
def cnn_model(x_train_data, keep_rate=0.7, seed=None):
    
    with tf.name_scope("layer_a"):
        # conv => 16*16*16
        conv1 = tf.layers.conv3d(inputs=x_train_data, filters=16, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
        # conv => 16*16*16
        conv2 = tf.layers.conv3d(inputs=conv1, filters=32, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
        # pool => 8*8*8
        pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[2, 2, 2], strides=2)
        
    with tf.name_scope("layer_c"):
        # conv => 8*8*8
        conv4 = tf.layers.conv3d(inputs=pool3, filters=64, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
        # conv => 8*8*8
        conv5 = tf.layers.conv3d(inputs=conv4, filters=128, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
        # pool => 4*4*4
        #pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[2, 2, 2], strides=2)
        
        
    with tf.name_scope("batch_norm"):
        cnn3d_bn = tf.layers.batch_normalization(inputs=conv5, training=True)
        
    with tf.name_scope("fully_con"):
        flattening = tf.layers.flatten(cnn3d_bn)#tf.reshape(cnn3d_bn, [-1, 4*4*4*128])
        dense = tf.layers.dense(inputs=flattening, units=128, activation=tf.nn.relu)
        # (1-keep_rate) is the probability that the node will be kept
        dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=True)
        
    with tf.name_scope("y_conv"):
        y_conv = tf.layers.dense(inputs=dropout, units=10)
    
    return y_conv

In [7]:
def train_neural_network(x_train_data, y_train_data, x_test_data, y_test_data, learning_rate=0.05, keep_rate=0.7, epochs=10, batch_size=128):

    gpu_number = 0
    start_config(gpu_number)
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True
    
    with tf.name_scope("cross_entropy"):
        prediction = cnn_model(x_input, keep_rate, seed=1)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input))
        saver = tf.train.Saver()
                              
    with tf.name_scope("training"):
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
           
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    iterations = int(len(x_train_data)/batch_size) + 1
    
    with tf.Session(config=config) as sess:
    #with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        import datetime

        start_time = datetime.datetime.now()

        iterations = int(len(x_train_data)/batch_size) + 1
        # run epochs
        for epoch in range(epochs):
            start_time_epoch = datetime.datetime.now()
            print('Epoch', epoch, 'started', end='')
            epoch_loss = 0
            # mini batch
            for itr in range(iterations):
                mini_batch_x = x_train_data[itr*batch_size: (itr+1)*batch_size]
                mini_batch_y = y_train_data[itr*batch_size: (itr+1)*batch_size]
                _optimizer, _cost = sess.run([optimizer, cost], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
                epoch_loss += _cost

            #  using mini batch in case not enough memory
            acc = 0
            itrs = int(len(x_test_data)/batch_size) + 1
            for itr in range(itrs):
                mini_batch_x_test = x_test_data[itr*batch_size: (itr+1)*batch_size]
                mini_batch_y_test = y_test_data[itr*batch_size: (itr+1)*batch_size]
                acc += sess.run(accuracy, feed_dict={x_input: mini_batch_x_test, y_input: mini_batch_y_test})

            end_time_epoch = datetime.datetime.now()
            print(' Testing Set Accuracy:',acc/itrs, ' Time elapse: ', str(end_time_epoch - start_time_epoch))

        end_time = datetime.datetime.now()

        print('Time elapse: ', str(end_time - start_time))

In [8]:
with tf.name_scope('inputs'):
    x_input = tf.placeholder(tf.float32, shape=[None, 2, 224, 224, 3])
    y_input = tf.placeholder(tf.float32, shape=[None, n_classes]) 

In [9]:
x_train_temp = []
for i in range(100):
    x_train_temp.append(np.random.rand(2,224,224,3))
    
x_train_temp = np.asarray(x_train_temp)
print(x_train_temp.shape)

print(y_train[:10].shape)

(100, 2, 224, 224, 3)
(10, 10)


In [10]:
train_neural_network(x_train_temp, 
                     y_train[:100], 
                     x_train_temp[:100], 
                     y_test[:100], 
                     epochs=10, 
                     batch_size=32, 
                     learning_rate=0.0001)

Configuring environment
Epoch 0 started Testing Set Accuracy: 0.09375  Time elapse:  0:00:04.155382
Epoch 1 started Testing Set Accuracy: 0.0546875  Time elapse:  0:00:01.466104
Epoch 2 started Testing Set Accuracy: 0.0390625  Time elapse:  0:00:01.465973
Epoch 3 started Testing Set Accuracy: 0.0703125  Time elapse:  0:00:01.477866
Epoch 4 started Testing Set Accuracy: 0.0390625  Time elapse:  0:00:01.468858
Epoch 5 started Testing Set Accuracy: 0.078125  Time elapse:  0:00:01.469577
Epoch 6 started Testing Set Accuracy: 0.0625  Time elapse:  0:00:01.469011
Epoch 7 started Testing Set Accuracy: 0.0625  Time elapse:  0:00:01.472653
Epoch 8 started Testing Set Accuracy: 0.0546875  Time elapse:  0:00:01.476749
Epoch 9 started Testing Set Accuracy: 0.0703125  Time elapse:  0:00:01.476717
Time elapse:  0:00:17.401226


In [11]:
x_train[:10].shape

(10, 2, 16, 16, 3)